# Track the International Space Station

##### Import Modules

In [1]:
import requests, json, time
import plotly.express as px
import argparse
from matplotlib.animation import FuncAnimation
import pandas as pd

##### Argparse

In [2]:
parser=argparse.ArgumentParser(description='Track and graph ISS position.')
parser.add_argument('-d','--duration',type=int,metavar='',
                    help='Total tracking duration.')
parser.add_argument('-s','--sample-freq',type=int,metavar='',
                    help='Interval between ISS location retrieval')
parser.add_argument('-p','--projection',type=str,metavar='',
                   help='Projection type of map.')
parser.add_argument('-f','--file',type=str,metavar='',
                   help='File name to save data to.')
parser.add_argument('-a','--animation',type=int,metavar='',
                   help='Animated plot.')
args=parser.parse_args()


##### Function to write to file

In [3]:
def file_io(mode,f_name,json_data=None):
    if mode == 'a+':
        with open(f_name,mode) as f_handler:
            json.dump(json_data,f_handler)
            f_handler.write("\n")
    elif mode == 'r' :
        with open(f_name,mode) as f_handler:
            f_content = f_handler.readlines()
            return f_content

##### Function to make a call to ISS open-API

In [4]:
def get_iss_info(f_name='default',write_optn='n',t_interval=1,t_total=1):
    print("Total number of data points: {}".format(int(t_total/t_interval)))
    t_counter = 0
    t_now = 1
    info_lst = []
    while t_now <= t_total:
        iss_data = requests.get('http://api.open-notify.org/iss-now.json').json()
        t_now += t_interval
        t_counter += 1
        if write_optn == 'y':
            mode = 'a+'
            file_io(mode,f_name,iss_data)
        else :
            print(iss_data)
        time.sleep(t_interval)
    return 0
    

##### Main

In [7]:
# get_iss_info(args.file,'y',args.sample-freq,args.duration)
get_iss_info('test','y',1,1)

f_content = file_io('r','test')

iss_data_lst = []
lat_lst = []
lon_lst = []
time_lst = []

for line in f_content:
    iss_data_lst.append(json.loads(line.strip()))

time_zero = iss_data_lst[0]['timestamp']

for element in iss_data_lst:
    lat_lst.append(element['iss_position']['latitude'])
    lon_lst.append(element['iss_position']['longitude'])
    time_lst.append((element['timestamp']-time_zero)/60)
print("Data collection complete.")

df=pd.DataFrame(zip(lat_lst,lon_lst,time_lst),columns=['Latitude','Longitude','Time'])

projection_mode = 'orthographic'
fig = px.scatter_geo(lat=lat_lst,lon=lon_lst,projection=projection_mode,
                     opacity=0.5,hover_name=time_lst)
fig.update_layout(width=850, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig = px.scatter_geo(lat=lat_lst,lon=lon_lst,
                     opacity=1,hover_name=time_lst,animation_frame=time_lst)
fig.update_layout(width=850, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Total number of data points: 1
Data collection complete.


##### Live tracking

##### Help

In [6]:

#print(type(go.Figure()))
#print(type(go.Scattergeo()))
#dir(go.Scattergeo())
fig = go.Figure(px.scatter_geo(lat=lat_lst,lon=lon_lst,projection='orthographic'))
fig.update_layout(height=400, margin={"r":0,"t":0,"l":0,"b":0})
print(type(fig))
fig.show()

fig = px.scatter_geo(lat=lat_lst,lon=lon_lst,projection=projection_mode,
                               opacity=0.5,hover_name=time_lst)
fig.update_layout(height=400, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

NameError: name 'go' is not defined